### How to Find a Serial Killer with Data. A Steps to Cleaning, Wrangling, and Feature Engineering.

Use Case:

Understanding crime patterns and trends is crucial for making data-driven decisions in law enforcement and policy-making. For this demonstration, we’ll leverage the Murder Accountability Project’s extensive dataset, which includes homicide information spanning decades in the United States. Our goal is to make this data clean, complete, and informative — suitable for predictive tasks like clearance rate prediction, offender profiling, and hotspot analysis. Each step we take in transforming this data will be designed to answer specific questions and create actionable insights for real-world applications

In [3]:
# Step 1: Load the Data

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('https://github.com/fenago/datasets/raw/refs/heads/main/SHR65_23.zip')
df.head()

,ID,CNTYFIPS,Ori,State,Agency,Agentype,Source,Solved,Year,Month,...,OffRace,OffEthnic,Weapon,Relationship,Circumstance,Subcircum,VicCount,OffCount,FileDate,MSA
0,197603001AK00101,"Anchorage, AK",AK00101,Alaska,Anchorage,Municipal police,FBI,Yes,1976,March,...,Black,Unknown or not reported,"Handgun - pistol, revolver, etc",Relationship not determined,Other arguments,NaN,0,0,30180.0,"Anchorage, AK"
1,197604001AK00101,"Anchorage, AK",AK00101,Alaska,Anchorage,Municipal police,FBI,Yes,1976,April,...,White,Unknown or not reported,"Handgun - pistol, revolver, etc",Girlfriend,Other arguments,NaN,0,0,30180.0,"Anchorage, AK"
2,197606001AK00101,"Anchorage, AK",AK00101,Alaska,Anchorage,Municipal police,FBI,Yes,1976,June,...,Black,Unknown or not reported,"Handgun - pistol, revolver, etc",Stranger,Other,NaN,0,0,30180.0,"Anchorage, AK"
3,197606002AK00101,"Anchorage, AK",AK00101,Alaska,Anchorage,Municipal police,FBI,Yes,1976,June,...,White,Unknown or not reported,"Handgun - pistol, revolver, etc",Other - known to victim,Other arguments,NaN,0,0,30180.0,"Anchorage, AK"
4,197607001AK00101,"Anchorage, AK",AK00101,Alaska,Anchorage,Municipal police,FBI,Yes,1976,July,...,American Indian or Alaskan Native,Unknown or not reported,Knife or cutting instrument,Brother,Other arguments,NaN,0,0,30180.0,"Anchorage, AK"


In [4]:
# Step 2: Generate Data Quality Report 1 (DQR1)
def generate_dqr1(df):
    # Initialize an empty DataFrame to store DQR1 results
    dqr1_df = pd.DataFrame(index=df.columns)
    
    # Basic Column Statistics
    dqr1_df['Missing_Values'] = df.isnull().sum()
    dqr1_df['Unique_Values'] = df.nunique()
    
    # Sample Value Counts (stored as a list to avoid multiple columns issue)
    dqr1_df['Sample_Value_Counts'] = df.apply(lambda x: list(x.value_counts().head(3).to_dict().items()))

    # Separate numeric and non-numeric columns
    numeric_df = df.select_dtypes(include=[np.number])
    non_numeric_df = df.select_dtypes(exclude=[np.number])

    # Descriptive Statistics for Numeric Columns Only
    descriptive_stats = numeric_df.describe().T
    stats_columns = ['mean', 'std', 'min', '25%', '50%', '75%', 'max']
    for col in stats_columns:
        dqr1_df[col] = descriptive_stats[col]

    # Convert stats columns to object type for safe assignment
    dqr1_df[stats_columns] = dqr1_df[stats_columns].astype(object)
    
    # Mark irrelevant stats for non-numeric columns
    dqr1_df.loc[non_numeric_df.columns, stats_columns] = 'N/A'

    # Correlation Analysis for Numeric Columns Only
    corr_matrix = numeric_df.corr().abs()
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    highly_correlated = [column for column in upper_tri.columns if any(upper_tri[column] > 0.8)]
    dqr1_df['Highly_Correlated'] = dqr1_df.index.isin(highly_correlated)

    # Variance Analysis for Numeric Columns Only
    variances = numeric_df.var()
    dqr1_df['High_Variance'] = dqr1_df.index.map(lambda x: variances[x] > 1.0 if x in variances else 'N/A')
    dqr1_df['Low_Variance'] = dqr1_df.index.map(lambda x: variances[x] < 0.01 if x in variances else 'N/A')

    # Potential Index Columns
    dqr1_df['Potential_Index'] = dqr1_df['Unique_Values'] > len(df) * 0.9

    # Additional Column Type Info
    dqr1_df['Data_Type'] = df.dtypes

    # Display the complete Data Quality Report with all rows and columns
    pd.options.display.max_rows = None
    pd.options.display.max_columns = None
    display(dqr1_df)

    return dqr1_df

# Generate the Data Quality Report
dqr1_df = generate_dqr1(df)

,Missing_Values,Unique_Values,Sample_Value_Counts,mean,std,min,25%,50%,75%,max,Highly_Correlated,High_Variance,Low_Variance,Potential_Index,Data_Type
ID,0,852394,"[(197702001NY05904, 13), (199204001FL05003, 12...",N/A,N/A,N/A,N/A,N/A,N/A,N/A,False,N/A,N/A,True,object
CNTYFIPS,0,3079,"[(Los Angeles, CA, 55681), (New York, NY, 4780...",N/A,N/A,N/A,N/A,N/A,N/A,N/A,False,N/A,N/A,False,object
Ori,0,13442,"[(NY03030, 47790), (ILCPD00, 30509), (CA01942,...",N/A,N/A,N/A,N/A,N/A,N/A,N/A,False,N/A,N/A,False,object
State,0,51,"[(California, 129741), (Texas, 86288), (New Yo...",N/A,N/A,N/A,N/A,N/A,N/A,N/A,False,N/A,N/A,False,object
Agency,0,10015,"[(New York, 47790), (Chicago, 30509), (Los Ang...",N/A,N/A,N/A,N/A,N/A,N/A,N/A,False,N/A,N/A,False,object
Agentype,0,8,"[(Municipal police, 692613), (Sheriff, 150999)...",N/A,N/A,N/A,N/A,N/A,N/A,N/A,False,N/A,N/A,False,object
Source,0,2,"[(FBI, 855718), (MAP, 38918)]",N/A,N/A,N/A,N/A,N/A,N/A,N/A,False,N/A,N/A,False,object
Solved,0,2,"[(Yes, 632457), (No, 262179)]",N/A,N/A,N/A,N/A,N/A,N/A,N/A,False,N/A,N/A,False,object
Year,0,48,"[(1993, 24337), (1992, 23793), (2021, 23699)]",1998.868617,14.134767,1976.0,1987.0,1997.0,2011.0,2023.0,False,True,False,False,int64
Month,0,12,"[(July, 82444), (August, 80644), (December, 76...",N/A,N/A,N/A,N/A,N/A,N/A,N/A,False,N/A,N/A,False,object


In [5]:
# Step 3: Generate a Detailed Data Quality Report with Recommendations

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer, QuantileTransformer, OneHotEncoder, OrdinalEncoder
import pandas as pd

def generate_data_quality_report_analysis(dqr1_df):
    # Initialize a new DataFrame to store the analysis
    analysis_df = dqr1_df[['Data_Type']].copy()  # Start with only Data_Type for essential columns

    # Add columns for analysis
    analysis_df['Treat_Flag'] = False  # Flag indicating if treatment is needed
    analysis_df['Outliers'] = False    # Flag indicating if outliers are detected
    analysis_df['Scale'] = False       # Flag for scaling recommendation
    analysis_df['Encode'] = False      # Flag for encoding recommendation

    # New True/False columns
    analysis_df['Missing_Values_Flag'] = False
    analysis_df['Infinities_Flag'] = False
    analysis_df['Correct_Data_Type'] = True
    analysis_df['Correlated_Columns_Flag'] = False

    # Additional columns to be ordered last
    analysis_df['Potential_Issues'] = ""  # Column to describe potential issues
    analysis_df['Recommendations'] = ""  # Column to provide recommendations
    analysis_df['Severity'] = "Severity 3: Optional considerations for improved quality"  # Default severity

    # 1. Missing Values Analysis (including % missing by column)
    missing_cols = dqr1_df[dqr1_df['Missing_Values'] > 0]
    for col in missing_cols.index:
        missing_percentage = (missing_cols['Missing_Values'][col] / len(dqr1_df)) * 100
        analysis_df.loc[col, 'Missing_Values_Flag'] = True
        analysis_df.loc[col, 'Potential_Issues'] += f"Missing values: {missing_percentage:.2f}%"
        analysis_df.loc[col, 'Treat_Flag'] = True
        analysis_df.loc[col, 'Severity'] = (
            "Severity 1: Issues that must be addressed" if missing_percentage > 50
            else "Severity 2: Significant quality issues that impact model performance"
        )
        if missing_percentage > 30:
            analysis_df.loc[col, 'Recommendations'] += "Consider dropping or using advanced imputation if critical."
        else:
            analysis_df.loc[col, 'Recommendations'] += "Simple imputation recommended (mean/median for numeric, mode for categorical)."

    # Additional missing values analysis: missing by row
    row_missing_percentage = (dqr1_df.isnull().sum(axis=1) / len(dqr1_df.columns)) * 100
    analysis_df.loc[:, 'Potential_Issues'] += f"\nRows with missing values exceeding 20%: {sum(row_missing_percentage > 20)}"
    
    # 2. Unique Values Analysis and Curse of Dimensionality Check
    high_cardinality_cols = dqr1_df[dqr1_df['Potential_Index'] == True]
    for col in high_cardinality_cols.index:
        analysis_df.loc[col, 'Potential_Issues'] += "High cardinality (potential identifier)."
        analysis_df.loc[col, 'Treat_Flag'] = True
        analysis_df.loc[col, 'Recommendations'] += "Consider dropping or indexing if needed."
        analysis_df.loc[col, 'Severity'] = "Severity 2: Significant quality issues that impact model performance"

    # Check for potential curse of dimensionality for categorical columns with high unique values
    for col in analysis_df.index:
        if dqr1_df['Data_Type'][col] == 'object' and dqr1_df['Unique_Values'][col] > 50:
            analysis_df.loc[col, 'Potential_Issues'] += "High unique values; may suffer from curse of dimensionality on encoding."
            analysis_df.loc[col, 'Treat_Flag'] = True
            analysis_df.loc[col, 'Encode'] = True
            analysis_df.loc[col, 'Recommendations'] += "Consider using OrdinalEncoder, feature hashing, or get_dummies if appropriate."
            analysis_df.loc[col, 'Severity'] = "Severity 2: Significant quality issues that impact model performance"

    # 3. Descriptive Statistics Analysis, Outlier Detection, and Scaling Recommendations
    for col in analysis_df.index:
        if dqr1_df['mean'][col] != 'N/A':
            mean_val, min_val, max_val = dqr1_df['mean'][col], dqr1_df['min'][col], dqr1_df['max'][col]
            iqr = dqr1_df['75%'][col] - dqr1_df['25%'][col]
            lower_bound, upper_bound = dqr1_df['25%'][col] - 1.5 * iqr, dqr1_df['75%'][col] + 1.5 * iqr
            # Check for outliers
            if (max_val > upper_bound) or (min_val < lower_bound):
                analysis_df.loc[col, 'Outliers'] = True
                analysis_df.loc[col, 'Potential_Issues'] += " Outliers detected."
                analysis_df.loc[col, 'Recommendations'] += " Consider outlier treatment (e.g., capping, transformation)."
                analysis_df.loc[col, 'Severity'] = "Severity 2: Significant quality issues that impact model performance"
            # Scaling recommendation
            if max_val - min_val > 1000:
                analysis_df.loc[col, 'Scale'] = True
                analysis_df.loc[col, 'Potential_Issues'] += " High range detected."
                analysis_df.loc[col, 'Recommendations'] += " Consider scaling using StandardScaler or RobustScaler."
                analysis_df.loc[col, 'Severity'] = "Severity 2: Significant quality issues that impact model performance"
            # Check for infinities
            if min_val == float('-inf') or max_val == float('inf'):
                analysis_df.loc[col, 'Infinities_Flag'] = True
                analysis_df.loc[col, 'Potential_Issues'] += " Contains positive or negative infinity values."
                analysis_df.loc[col, 'Treat_Flag'] = True
                analysis_df.loc[col, 'Recommendations'] += " Consider handling infinity values by replacing or capping."
                analysis_df.loc[col, 'Severity'] = "Severity 1: Issues that must be addressed"
            else:
                # Additional scaler recommendations
                analysis_df.loc[col, 'Recommendations'] += " Alternative scaling options: MinMaxScaler, PowerTransformer, or QuantileTransformer."

    # 4. Variance Analysis
    low_variance_cols = dqr1_df[dqr1_df['Low_Variance'] == True]
    for col in low_variance_cols.index:
        analysis_df.loc[col, 'Potential_Issues'] += " Low variance detected."
        analysis_df.loc[col, 'Treat_Flag'] = True
        analysis_df.loc[col, 'Recommendations'] += " Consider dropping due to low variance."
        analysis_df.loc[col, 'Severity'] = "Severity 2: Significant quality issues that impact model performance"

    # 5. Correlation Analysis
    highly_correlated_cols = dqr1_df[dqr1_df['Highly_Correlated'] == True]
    for col in highly_correlated_cols.index:
        analysis_df.loc[col, 'Correlated_Columns_Flag'] = True
        analysis_df.loc[col, 'Potential_Issues'] += " High correlation with other columns."
        analysis_df.loc[col, 'Treat_Flag'] = True
        analysis_df.loc[col, 'Recommendations'] += " Consider removing redundant columns or using PCA."
        analysis_df.loc[col, 'Severity'] = "Severity 2: Significant quality issues that impact model performance"

    # 6. Data Type and Encoding Analysis
    for col in analysis_df.index:
        if analysis_df['Data_Type'][col] == 'object':
            analysis_df.loc[col, 'Correct_Data_Type'] = False
            analysis_df.loc[col, 'Potential_Issues'] += " Categorical data type detected."
            analysis_df.loc[col, 'Treat_Flag'] = True
            analysis_df.loc[col, 'Encode'] = True
            if dqr1_df['Unique_Values'][col] <= 10:
                analysis_df.loc[col, 'Recommendations'] += " Use OneHotEncoder or pd.get_dummies for encoding."
            else:
                analysis_df.loc[col, 'Recommendations'] += " Use OrdinalEncoder or feature hashing to handle high cardinality."
            analysis_df.loc[col, 'Severity'] = "Severity 2: Significant quality issues that impact model performance"
        elif pd.api.types.is_numeric_dtype(analysis_df['Data_Type'][col]) and dqr1_df['Potential_Index'][col] == True:
            analysis_df.loc[col, 'Potential_Issues'] += " Potential index with high cardinality."
            analysis_df.loc[col, 'Recommendations'] += " Consider converting to `category` type to save memory."
        elif analysis_df['Data_Type'][col] == 'object' and dqr1_df['Unique_Values'][col] > 50:
            analysis_df.loc[col, 'Potential_Issues'] += " Potential text column detected; consider for NLP analysis."
            analysis_df.loc[col, 'Treat_Flag'] = True
            analysis_df.loc[col, 'Recommendations'] += " Consider using NLP techniques for text analysis."
            analysis_df.loc[col, 'Severity'] = "Severity 2: Significant quality issues that impact model performance"

    # 7. Duplicate Check (removing unhashable types)
    hashable_df = dqr1_df.map(lambda x: x if isinstance(x, (str, int, float, bool, pd.Timestamp)) else None)
    duplicate_rows = hashable_df.duplicated().sum()
    if duplicate_rows > 0:
        analysis_df.loc[:, 'Potential_Issues'] += f"\nDataset contains {duplicate_rows} duplicate rows."
        analysis_df.loc[:, 'Treat_Flag'] = True
        analysis_df.loc[:, 'Recommendations'] += " Consider removing duplicate rows to improve data quality."
        analysis_df.loc[:, 'Severity'] = "Severity 1: Issues that must be addressed"

    # 8. Potential Index Columns Check
    potential_index_cols = [col for col in dqr1_df.index if dqr1_df['Unique_Values'][col] == len(dqr1_df)]
    if potential_index_cols:
        for col in potential_index_cols:
            analysis_df.loc[col, 'Potential_Issues'] += " Unique values for each row; could serve as index."
            analysis_df.loc[col, 'Recommendations'] += " Consider using this column as an index."
            analysis_df.loc[col, 'Severity'] = "Severity 3: Optional considerations for improved quality"

    # Sort by 'Severity' and 'Treat_Flag' for priority order
    analysis_df = analysis_df.sort_values(by=['Severity', 'Treat_Flag'], ascending=[True, False])

    # Reorder columns: True/False flags first, followed by analysis columns, and ending with `Data_Type`, `Potential_Issues`, and `Recommendations`
    column_order = [
        'Treat_Flag', 'Outliers', 'Scale', 'Encode', 'Missing_Values_Flag', 'Infinities_Flag',
        'Correct_Data_Type', 'Correlated_Columns_Flag', 'Severity', 'Data_Type', 'Potential_Issues', 'Recommendations'
    ]
    analysis_df = analysis_df[column_order]
    
    # Ensure Potential_Issues and Recommendations are fully readable
    pd.set_option('display.max_colwidth', None)
    pd.set_option('display.max_rows', None)
    
    # Display the Data Quality Report Analysis
    display(analysis_df)

    return analysis_df

# Generate the analysis report based on dqr1_df
analysis_df = generate_data_quality_report_analysis(dqr1_df)

,Treat_Flag,Outliers,Scale,Encode,Missing_Values_Flag,Infinities_Flag,Correct_Data_Type,Correlated_Columns_Flag,Severity,Data_Type,Potential_Issues,Recommendations
ID,True,False,False,True,False,False,False,False,Severity 1: Issues that must be addressed,object,\nRows with missing values exceeding 20%: 0High cardinality (potential identifier).High unique values; may suffer from curse of dimensionality on encoding. Categorical data type detected.\nDataset contains 8 duplicate rows.,"Consider dropping or indexing if needed.Consider using OrdinalEncoder, feature hashing, or get_dummies if appropriate. Use OrdinalEncoder or feature hashing to handle high cardinality. Consider removing duplicate rows to improve data quality."
CNTYFIPS,True,False,False,True,False,False,False,False,Severity 1: Issues that must be addressed,object,\nRows with missing values exceeding 20%: 0High unique values; may suffer from curse of dimensionality on encoding. Categorical data type detected.\nDataset contains 8 duplicate rows.,"Consider using OrdinalEncoder, feature hashing, or get_dummies if appropriate. Use OrdinalEncoder or feature hashing to handle high cardinality. Consider removing duplicate rows to improve data quality."
Ori,True,False,False,True,False,False,False,False,Severity 1: Issues that must be addressed,object,\nRows with missing values exceeding 20%: 0High unique values; may suffer from curse of dimensionality on encoding. Categorical data type detected.\nDataset contains 8 duplicate rows.,"Consider using OrdinalEncoder, feature hashing, or get_dummies if appropriate. Use OrdinalEncoder or feature hashing to handle high cardinality. Consider removing duplicate rows to improve data quality."
State,True,False,False,True,False,False,False,False,Severity 1: Issues that must be addressed,object,\nRows with missing values exceeding 20%: 0High unique values; may suffer from curse of dimensionality on encoding. Categorical data type detected.\nDataset contains 8 duplicate rows.,"Consider using OrdinalEncoder, feature hashing, or get_dummies if appropriate. Use OrdinalEncoder or feature hashing to handle high cardinality. Consider removing duplicate rows to improve data quality."
Agency,True,False,False,True,False,False,False,False,Severity 1: Issues that must be addressed,object,\nRows with missing values exceeding 20%: 0High unique values; may suffer from curse of dimensionality on encoding. Categorical data type detected.\nDataset contains 8 duplicate rows.,"Consider using OrdinalEncoder, feature hashing, or get_dummies if appropriate. Use OrdinalEncoder or feature hashing to handle high cardinality. Consider removing duplicate rows to improve data quality."
Agentype,True,False,False,True,False,False,False,False,Severity 1: Issues that must be addressed,object,\nRows with missing values exceeding 20%: 0 Categorical data type detected.\nDataset contains 8 duplicate rows.,Use OneHotEncoder or pd.get_dummies for encoding. Consider removing duplicate rows to improve data quality.
Source,True,False,False,True,False,False,False,False,Severity 1: Issues that must be addressed,object,\nRows with missing values exceeding 20%: 0 Categorical data type detected.\nDataset contains 8 duplicate rows.,Use OneHotEncoder or pd.get_dummies for encoding. Consider removing duplicate rows to improve data quality.
Solved,True,False,False,True,False,False,False,False,Severity 1: Issues that must be addressed,object,\nRows with missing values exceeding 20%: 0 Categorical data type detected.\nDataset contains 8 duplicate rows.,Use OneHotEncoder or pd.get_dummies for encoding. Consider removing duplicate rows to improve data quality.
Year,True,False,False,False,False,False,True,False,Severity 1: Issues that must be addressed,int64,\nRows with missing values exceeding 20%: 0\nDataset contains 8 duplicate rows.,"Alternative scaling options: MinMaxScaler, PowerTransformer, or QuantileTransformer. Consider removing duplicate rows to improve data quality."
Month,True,Fals